# Get the dataset


In [1]:
import csv
import numpy as np
import config

test_image = np.load(config.t_path)
test_names = np.load(config.n_path)

assert(test_image.shape[0] == test_names.shape[0])


/home/phuonghd/.conda/envs/nhatenv/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Boilerplate Code

In [2]:
import torch
from typing import Tuple
class Data(torch.utils.data.Dataset):
    def __init__(self, image: np.ndarray, name: np.ndarray) -> None:
        assert(image.shape[0] == name.shape[0])
        self.image = image
        self.name = name
    
    def __len__(self) -> int:
        return self.image.shape[0]
    
    def __getitem__(self, idx: int) -> 'Tuple[np.ndarray, str]':
        return self.image[idx], self.name[idx]


# Model

In [3]:
model = config.get_model()
model.load_state_dict(torch.load(config.model_path))

use_cuda = torch.cuda.is_available()
if not use_cuda:
    print("CUDA not used!")
device = torch.device("cuda" if use_cuda else "cpu")

model = model.to(device)


# Run tests

In [4]:
from tqdm import tqdm

tests = Data(test_image, test_names)
tests_dataloader = torch.utils.data.DataLoader(
    tests, batch_size = config.batch_size)


with torch.no_grad(), open("result.csv", "w", newline='') as result:
    result_writer = csv.writer(result)
    for image, names in tqdm(tests_dataloader):
        image = image.to(device, dtype=torch.float)

        output = model(image)
        output = output.argmax(dim=1).cpu().item()
        assert(output.shape[0] == names.shape[0])
        
        result_writer.writerows(zip(names, output))


  1%|          | 3/296 [00:03<04:11,  1.17it/s]

('frame_000000.PNG', 'frame_000001.PNG', 'frame_000002.PNG', 'frame_000003.PNG', 'frame_000004.PNG', 'frame_000005.PNG', 'frame_000006.PNG', 'frame_000007.PNG', 'frame_000008.PNG', 'frame_000009.PNG', 'frame_000010.PNG', 'frame_000011.PNG', 'frame_000012.PNG', 'frame_000013.PNG', 'frame_000014.PNG', 'frame_000015.PNG', 'frame_000016.PNG', 'frame_000017.PNG', 'frame_000018.PNG', 'frame_000019.PNG', 'frame_000020.PNG', 'frame_000021.PNG', 'frame_000022.PNG', 'frame_000023.PNG', 'frame_000024.PNG', 'frame_000025.PNG', 'frame_000026.PNG', 'frame_000027.PNG', 'frame_000028.PNG', 'frame_000029.PNG', 'frame_000030.PNG', 'frame_000031.PNG') tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1], device='cuda:0')
('frame_000032.PNG', 'frame_000033.PNG', 'frame_000034.PNG', 'frame_000035.PNG', 'frame_000036.PNG', 'frame_000037.PNG', 'frame_000038.PNG', 'frame_000039.PNG', 'frame_000040.PNG', 'frame_000041.PNG', 'frame_000042.PNG', 'frame_0

  2%|▏         | 5/296 [00:03<02:09,  2.25it/s]

('frame_000096.PNG', 'frame_000097.PNG', 'frame_000098.PNG', 'frame_000099.PNG', 'frame_000100.PNG', 'frame_000101.PNG', 'frame_000102.PNG', 'frame_000103.PNG', 'frame_000104.PNG', 'frame_000105.PNG', 'frame_000106.PNG', 'frame_000107.PNG', 'frame_000108.PNG', 'frame_000109.PNG', 'frame_000110.PNG', 'frame_000111.PNG', 'frame_000112.PNG', 'frame_000113.PNG', 'frame_000114.PNG', 'frame_000115.PNG', 'frame_000116.PNG', 'frame_000117.PNG', 'frame_000118.PNG', 'frame_000119.PNG', 'frame_000120.PNG', 'frame_000121.PNG', 'frame_000122.PNG', 'frame_000123.PNG', 'frame_000124.PNG', 'frame_000125.PNG', 'frame_000126.PNG', 'frame_000127.PNG') tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1], device='cuda:0')
('frame_000128.PNG', 'frame_000129.PNG', 'frame_000130.PNG', 'frame_000131.PNG', 'frame_000132.PNG', 'frame_000133.PNG', 'frame_000134.PNG', 'frame_000135.PNG', 'frame_000136.PNG', 'frame_000137.PNG', 'frame_000138.PNG', 'frame_0

  4%|▎         | 11/296 [00:03<00:43,  6.53it/s]

...

('frame_001963.PNG', 'frame_001964.PNG', 'frame_001965.PNG', 'frame_001966.PNG', 'frame_001967.PNG', 'frame_001968.PNG', 'frame_001969.PNG', 'frame_001970.PNG', 'frame_001971.PNG', 'frame_001972.PNG', 'frame_001973.PNG', 'frame_001974.PNG', 'frame_001975.PNG', 'frame_001976.PNG', 'frame_001977.PNG', 'frame_001978.PNG', 'frame_001979.PNG', 'frame_001980.PNG', 'frame_001981.PNG', 'frame_001982.PNG', 'frame_001983.PNG', 'frame_001984.PNG', 'frame_001985.PNG', 'frame_001986.PNG', 'frame_001987.PNG', 'frame_001988.PNG', 'frame_001989.PNG', 'frame_001990.PNG', 'frame_001991.PNG', 'frame_001992.PNG', 'frame_001993.PNG', 'frame_001994.PNG') tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1], device='cuda:0')
('frame_001995.PNG', 'frame_001996.PNG', 'frame_001997.PNG', 'frame_001998.PNG', 'frame_001999.PNG', 'frame_002000.PNG', 'frame_002001.PNG', 'frame_002002.PNG', 'frame_002003.PNG', 'frame_002004.PNG', 'frame_002005.PNG', 'frame_0

 98%|█████████▊| 290/296 [00:14<00:00, 22.87it/s]

tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1], device='cuda:0')
('frame_002194.PNG', 'frame_002195.PNG', 'frame_002196.PNG', 'frame_002197.PNG', 'frame_002198.PNG', 'frame_002199.PNG', 'frame_002200.PNG', 'frame_002201.PNG', 'frame_002202.PNG', 'frame_002203.PNG', 'frame_002204.PNG', 'frame_002205.PNG', 'frame_002206.PNG', 'frame_002207.PNG', 'frame_002208.PNG', 'frame_002209.PNG', 'frame_002210.PNG', 'frame_002211.PNG', 'frame_002212.PNG', 'frame_002213.PNG', 'frame_002214.PNG', 'frame_002215.PNG', 'frame_002216.PNG', 'frame_002217.PNG', 'frame_002218.PNG', 'frame_002219.PNG', 'frame_002220.PNG', 'frame_002221.PNG', 'frame_002222.PNG', 'frame_002223.PNG', 'frame_002224.PNG', 'frame_002225.PNG') tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1], device='cuda:0')
('frame_002226.PNG', 'frame_002227.PNG', 'frame_002228.PNG', 'frame_002229.PNG', 'frame_002230.PNG'

100%|██████████| 296/296 [00:14<00:00, 19.78it/s]

tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1], device='cuda:0')
('frame_002457.PNG', 'frame_002458.PNG', 'frame_002459.PNG', 'frame_002460.PNG', 'frame_002461.PNG', 'frame_002462.PNG', 'frame_002463.PNG', 'frame_002464.PNG', 'frame_002465.PNG', 'frame_002466.PNG', 'frame_002467.PNG', 'frame_002468.PNG', 'frame_002469.PNG', 'frame_002470.PNG', 'frame_002471.PNG', 'frame_002472.PNG', 'frame_002473.PNG', 'frame_002474.PNG', 'frame_002475.PNG', 'frame_002476.PNG', 'frame_002477.PNG', 'frame_002478.PNG', 'frame_002479.PNG', 'frame_002480.PNG', 'frame_002481.PNG', 'frame_002482.PNG', 'frame_002483.PNG', 'frame_002484.PNG', 'frame_002485.PNG', 'frame_002486.PNG', 'frame_002487.PNG', 'frame_002488.PNG') tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1], device='cuda:0')
('frame_002489.PNG', 'frame_002490.PNG', 'frame_002491.PNG', 'frame_002492.PNG', 'frame_002539.PNG'